In [3]:
'''
    20251201:
        开放问题回答，生成--for 评测

    20251208:
        生成judge的文件
'''

'\n    20251201:\n        开放问题回答，生成--for 评测\n\n    20251208:\n        生成judge的文件\n'

In [1]:
import datasets

In [48]:
import transformers

import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# download checkpoint
from accelerate import load_checkpoint_and_dispatch
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

#from accelerate import load_checkpoint_and_dispatch

from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

import util
import importlib

importlib.reload(util)      # 只能 reload 模块本身
from util import calc_ppl, get_test_data   # reload 后再重新 import 函数

import pandas as pd
import copy

In [4]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='cuda'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

In [5]:
'''
load test set

'''

'\nload test set\n\n'

In [8]:
ds_test = datasets.load_dataset('json', data_files ='/root/autodl-fs/LRP/open_ended_dataset/all_data.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
ds_test['train'][121]

{'text': 'Điều gì sẽ xảy ra nếu người Aztec đã thành công trong việc đẩy lùi các chinh phục viên Tây Ban Nha?',
 'lang': 'vi',
 '__index_level_0__': 51}

# 20251208

In [110]:
ds_lape = datasets.load_dataset('json', data_files='/root/autodl-fs/LRP/open_ended_data_generation/20251201_LAPE_org_generation.json')

In [111]:
df_lape = ds_lape['train'].to_pandas().loc[df_lape['model_type']!='org_model|open_ended'].reset_index(drop=True)

In [112]:
df_lape.head()

,text,lang,__index_level_0__,answer,model_type
0,How can I improve my time management skills?,en,0,Time management is the process of planning and...,mask_en|open_ended
1,What are the most effective ways to deal with ...,en,1,"Stress is a normal part of life, but chronic s...",mask_en|open_ended
2,What are the main differences between Python a...,en,2,Python and JavaScript are two popular programm...,mask_en|open_ended
3,How can I increase my productivity while worki...,en,3,Here are some tips to help you increase your p...,mask_en|open_ended
4,Can you explain the basics of quantum computing?,en,4,Quantum computing is a rapidly growing field t...,mask_en|open_ended


In [113]:
df_lape['model_type'].value_counts()

model_type
mask_en|open_ended    210
mask_vi|open_ended    210
mask_zh|open_ended    210
Name: count, dtype: int64

In [114]:
# new 

In [115]:
ds_lrp = datasets.load_dataset('json', data_files='/root/autodl-fs/LRP/open_ended_data_generation/20251204_all_exp/1208_all.json')

In [116]:
df_lrp = ds_lrp['train'].to_pandas()

In [117]:
ds_lrp

DatasetDict({
    train: Dataset({
        features: ['text', 'lang', '__index_level_0__', 'answer', 'model_type'],
        num_rows: 48930
    })
})

In [118]:
df_all = pd.concat([df_lape, df_lrp]).drop_duplicates(subset=['text', 'lang', 'answer', 'model_type']).reset_index(drop=True)

In [119]:
df_all['model_type'].value_counts()

model_type
org_model|open_ended                                       210
mask_en|open_ended                                         210
gap_rate_top_0_5perc_LRP_kur_res_zscore_zh|open_ended      210
gap_rate_top_1perc_LRP_kur_res_en|open_ended               210
gap_rate_top_1perc_LRP_kur_res_vi|open_ended               210
                                                          ... 
gaprate_th_11_selected_LRP_kur_res_vi_zscore|open_ended    210
gaprate_th_11_selected_LRP_kur_res_zh_zscore|open_ended    210
gaprate_th_14_selected_LRP_kur_res_en_zscore|open_ended    210
gaprate_th_14_selected_LRP_kur_res_vi_zscore|open_ended    210
gaprate_th_14_selected_LRP_kur_res_zh_zscore|open_ended    210
Name: count, Length: 154, dtype: int64

In [123]:
len(df_all)/210

154.0

In [124]:
'''
生成judge的prompt

'''
def generate_judge_file(df, save_path):
    prompt_template = (
        "You are a neutral judge. Score the model’s answer from 1 to 10 based on correctness, completeness, clarity, and usefulness.\n"
        "Respond ONLY in this JSON format:\n"
        '{{"score": <1-10>, "reason": "<brief reason>"}}\n\n'
        "Question:\n{question}\n\n"
        "Answer:\n{answer}"
    )
    
    def get_judge_promt(x):
        return prompt_template.format(question= x[0], answer= x[1])
    df['judge_prompt'] = df[['text','answer']].apply(lambda x: prompt_template.format(question= x[0], answer= x[1]), axis=1)
    ds_final = datasets.Dataset.from_pandas(df)
    
    #ds_final.to_json('~/autodl-fs/LRP/open_ended_data_generation/20251201_LRP_based_generation4judge.json',index=False)
    ds_final.to_json(save_path, index=False)


In [125]:
generate_judge_file(df_all, '/root/autodl-fs/LRP/open_ended_data_generation/20251204_all_exp/1208_all_generation4judge.json')

/tmp/ipykernel_2146/483060424.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['judge_prompt'] = df[['text','answer']].apply(lambda x: prompt_template.format(question= x[0], answer= x[1]), axis=1)


Creating json from Arrow format:   0%|          | 0/33 [00:00<?, ?ba/s]

In [23]:
# 分析结果

In [76]:
judege_path ='/root/autodl-fs/LRP/open_ended_data_generation/20251204_all_exp/1208_some_v2_generation4judge.json_gpt4o.json'

In [77]:
ds_judge_res = datasets.load_dataset('json', data_files = judege_path)




Generating train split: 0 examples [00:00, ? examples/s]

In [78]:
ds_judge_res['train'][0]

{'text': 'How can I improve my time management skills?',
 'lang': 'en',
 '__index_level_0__': 0,
 'answer': "Time management is the process of planning and controlling the amount of time spent on different activities to maximize efficiency and productivity. Here are some tips that can help you improve your time management skills:\n\n1. Set clear goals: Setting clear goals helps you prioritize tasks and allocate your time effectively. Make sure your goals are specific, measurable, achievable, relevant, and time-bound (SMART).\n2. Create a schedule: Plan out your day, week, or month in advance using a calendar or planner. Schedule fixed and flexible tasks, and make sure to include time for breaks and relaxation.\n3. Prioritize tasks: Determine which tasks are most important and need to be completed first. Use the Eisenhower Matrix to categorize tasks into urgent vs. important, and focus on the most critical ones first.\n4. Avoid multitasking: Multitasking can reduce productivity and incr

In [79]:
def get_analysis_res(ds_judge_res):
    df_judge_res = ds_judge_res['train'].to_pandas()
    # get score
    def get_score(x):
        try:
            return  json.loads(x)['score']
        except:
            return -1
    
    df_judge_res['score'] = df_judge_res['output'].apply(get_score)

    df_judge_res['class_tpye'] = df_judge_res['lang']+'|' + df_judge_res['model_type']

    result_dict = {}
    for imodel_type in list(df_judge_res['class_tpye'].value_counts().keys()):
        print('='*20)
        print('imodel_type:', imodel_type)
        df_tmp = df_judge_res.loc[df_judge_res['class_tpye'] == imodel_type].reset_index(drop=True)
        df_tmp = df_tmp.loc[df_tmp['score']!=-1].reset_index(drop=True)
        print('valid count:', len(df_tmp))
        result_dict[imodel_type] = df_tmp['score'].mean()
              
    return result_dict


In [80]:
result_dict = get_analysis_res(ds_judge_res)

imodel_type: en|org_model|open_ended
valid count: 139
imodel_type: vi|org_model|open_ended
valid count: 139
imodel_type: zh|org_model|open_ended
valid count: 140
imodel_type: vi|gap_rate_top_0_1_perc_LRP_kur_res_en|open_ended
valid count: 69
imodel_type: zh|gap_rate_top_0_1_perc_LRP_kur_res_en|open_ended
valid count: 70
imodel_type: en|gap_rate_top_0_1_perc_LRP_kur_res_vi|open_ended
valid count: 69
imodel_type: vi|gap_rate_top_0_1_perc_LRP_kur_res_vi|open_ended
valid count: 70
imodel_type: zh|gap_rate_top_0_1_perc_LRP_kur_res_vi|open_ended
valid count: 70
imodel_type: en|gap_rate_top_0_1_perc_LRP_kur_res_zh|open_ended
valid count: 69
imodel_type: vi|gap_rate_top_0_1_perc_LRP_kur_res_zh|open_ended
valid count: 69
imodel_type: zh|gap_rate_top_0_1_perc_LRP_kur_res_zh|open_ended
valid count: 70
imodel_type: en|gap_rate_top_0_1_perc_LRP_kur_res_zscore_en|open_ended
valid count: 70
imodel_type: vi|gap_rate_top_0_1_perc_LRP_kur_res_zscore_en|open_ended
valid count: 69
imodel_type: zh|gap_rate

In [81]:
result_dict

{'en|org_model|open_ended': 8.028776978417266,
 'vi|org_model|open_ended': 7.179856115107913,
 'zh|org_model|open_ended': 6.742857142857143,
 'vi|gap_rate_top_0_1_perc_LRP_kur_res_en|open_ended': 7.057971014492754,
 'zh|gap_rate_top_0_1_perc_LRP_kur_res_en|open_ended': 7.142857142857143,
 'en|gap_rate_top_0_1_perc_LRP_kur_res_vi|open_ended': 7.884057971014493,
 'vi|gap_rate_top_0_1_perc_LRP_kur_res_vi|open_ended': 6.885714285714286,
 'zh|gap_rate_top_0_1_perc_LRP_kur_res_vi|open_ended': 7.0285714285714285,
 'en|gap_rate_top_0_1_perc_LRP_kur_res_zh|open_ended': 7.913043478260869,
 'vi|gap_rate_top_0_1_perc_LRP_kur_res_zh|open_ended': 7.231884057971015,
 'zh|gap_rate_top_0_1_perc_LRP_kur_res_zh|open_ended': 6.857142857142857,
 'en|gap_rate_top_0_1_perc_LRP_kur_res_zscore_en|open_ended': 7.757142857142857,
 'vi|gap_rate_top_0_1_perc_LRP_kur_res_zscore_en|open_ended': 7.159420289855072,
 'zh|gap_rate_top_0_1_perc_LRP_kur_res_zscore_en|open_ended': 6.928571428571429,
 'en|gap_rate_top_0_1_p

In [82]:
import pandas as pd
import numpy as np
import re

# ========= 1. 粘贴你的数据 =========
res = result_dict

# ========= 2. 格式化解析 =========
rows=[]
for k,v in res.items():
    lang, method, _ = k.split("|")
    rows.append([method, lang, float(v)])
df = pd.DataFrame(rows, columns=["method","lang","score"])

# ========= 3. org baseline =========
org = df[df.method=="org_model"].pivot(index="method",columns="lang",values="score").iloc[0]

# ========= 4. pivot为宽表 =========
table = df.pivot(index="method",columns="lang",values="score")
table = table[["en","vi","zh"]]  # fix顺序

# ========= 5. drop =========
drop = (org - table)

# ========= 6. target language提取 =========
def get_target(m):
    # 优先从方法名抓最后一个语言标记
    mm = re.findall(r'_(en|vi|zh)\b', m.lower())
    if mm:
        return mm[-1]
    # mask_en格式
    if m.startswith("mask_"):
        return m.split("_")[1]
    return None

drop["target"] = drop.index.map(get_target)

# ========= 7. specificity & stability =========
spec=[]; stab=[]; score=[]
for idx,row in drop.iterrows():
    tgt=row["target"]
    # 无target语言的跳过
    if tgt not in ["en","vi","zh"]:
        spec.append(np.nan); stab.append(np.nan); score.append(np.nan)
        continue
    d_target = row[tgt]
    d_other = [row[x] for x in ["en","vi","zh"] if x!=tgt]
    specificity = d_target / (np.mean(d_other) + 1e-8)
    stability = np.std([row["en"],row["vi"],row["zh"]])
    s = specificity / (1 + stability)
    spec.append(specificity); stab.append(stability); score.append(s)

drop["specificity"]=spec
drop["stability"]=stab
drop["score"]=score

# ========= 8. 排序 & 输出 =========
ranking = drop.sort_values("score", ascending=False)
print(ranking[["target","en","vi","zh","specificity","stability","score"]])

# ========= 9. 保存CSV =========
ranking.to_csv("full_ranking.csv")
print("\nSaved to full_ranking.csv")


lang                                         target        en        vi  \
method                                                                    
gap_rate_top_0_5perc_LRP_kur_res_zscore_vi       vi -0.014701  0.351285   
gaprate_th_0.99_selected_LRP_kur_res_vi          vi  0.128777  0.636999   
gap_rate_top_0_5perc_LRP_kur_res_vi              vi  0.114491  0.236999   
gap_rate_bottom_0_0_1perc_LRP_kur_res_zh         zh -0.085509  0.051285   
mask_vi                                          vi  0.058189 -0.211448   
...                                             ...       ...       ...   
gap_rate_top_0_1_perc_LRP_kur_res_zh             zh  0.115734 -0.052028   
gap_rate_top_0_1_perc_LRP_kur_res_vi             vi  0.144719  0.294142   
gap_rate_bottom_0_1_perc_LRP_kur_res_vi          vi  0.100206 -0.008550   
gaprate_th_11_selected_LRP_kur_res_en_zscore   None  1.100206  3.636999   
org_model                                      None  0.000000  0.000000   

lang                    